<a href="https://colab.research.google.com/github/lagom-QB/M11/blob/master/20%7C05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------------------------------------------------------
Update training and test function in such a way that train loss is ploted on "train_loss" graph, test loss on "test_loss" graph and accuracy on the test set on "test_accuracy" graph.

In [9]:
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch
from torch.nn.modules import loss
from torchvision import datasets, transforms


train_dataset = datasets.MNIST('/data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
test_dataset = datasets.MNIST('../data', train=False, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

class MySGD(torch.optim.Optimizer):  
  def __init__(self, params, lr):
    self._lr = lr
    defaults = {}
    super().__init__(params, defaults)

  @torch.no_grad()
  def step(self):
    for group in self.param_groups:
      for p in group['params']:
        if p.grad is None:
          continue
        d_p = p.grad
        p.add_(d_p, alpha=-self._lr)
    return loss

class Flatten(nn.Module):
  def forward(self, x):
    batch_size = x.shape[0]
    return x.view(batch_size, -1)

def get_model():
  return nn.Sequential(Flatten(), 
                       nn.Linear(784, 512), 
                       nn.Tanh(),
                       nn.Linear(512, 64), 
                       nn.Tanh(),
                       nn.Linear(64, 10))

def train_writer(writer, model, train_loader, optimizer, loss_function, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        writer.add_scalar("train_loss", torch.tensor(loss.item()), global_step = epoch)
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test_writer(writer, model, test_loader, loss_function):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += loss_function(output, target).sum().item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            writer.add_scalar("test_accuracy", torch.tensor(correct), global_step = epoch)

    test_loss /= len(test_loader.dataset)
    writer.add_scalar("test_loss", torch.tensor(test_loss), global_step = epoch)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

loss_function = loss.CrossEntropyLoss()
writer = SummaryWriter(log_dir="ass_1_logs")
model = get_model()
optim = MySGD(model.parameters(), lr=0.1)

for epoch in range(1, 5):
  train_writer(writer, model, train_loader, optim, loss_function, epoch)
  test_writer(writer, model, test_loader, loss_function)
writer.close()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.324755
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.472108
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.423116
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.264435
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.343311

Test set: Average loss: 0.0031, Accuracy: 9425/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.291894
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.112025
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.113538
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.143097
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.085854

Test set: Average loss: 0.0020, Accuracy: 9608/10000 (96%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.183233
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.038941
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.073978
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.013608
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.132436

Test set: Average loss: 0.0014, Accuracy: 9728/10000 (97%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.081560
Train Epoch: 4 [12800/60000 (21%)]	Lo

In [11]:
%load_ext tensorboard
%tensorboard --logdir ass_1_logs

<IPython.core.display.Javascript object>